In [40]:
import yfinance as yf
import numpy as np
import pandas as pd
from get_data_yf import get_symbols

#TODO writes advised company recommendation data to an excel file

RECOMMENDATION_WEIGHTS = [2, 1, 0, -1, -2] #weights for strongBuy, buy, hold, sell,	strongSell advisors respectively
TIMEZONE_WEIGHTS = [4, 3, 2, 1] # weights for 0, -1, -2, -3rd months respectively
ADVICE_COUNT = 50 #how many of best scored symbols we want to get data
file_path = 'data/recommendations'

In [ ]:
symbols = get_symbols()
symbols.append('FB')
symbol = yf.Ticker('Fafsqwffqwwqfa')
recommendations = symbol.recommendations
#recommendations = recommendations.drop('period', axis='columns')
print((recommendations.empty))

In [ ]:
def create_weigth_kernel(vertical_weights, horizontal_weights):
    kernel_values = np.outer(vertical_weights, horizontal_weights)
    kernel = pd.DataFrame(kernel_values)
    kernel.rename(columns={0:'strongBuy', 1:'buy', 2:'hold', 
                              3:'sell', 4:'strongSell'}, inplace = True)
    
    return kernel

In [ ]:
def recommendation_score(recommendations, weight_kernel):
    #returns score and how many anaylst we have as advisor in total
    sum = recommendations.to_numpy().sum()
    if sum == 0:
        return None, None
    weighted_sum = (weight_kernel * recommendations).to_numpy().sum()
    return weighted_sum / sum, sum 

In [ ]:
def evaluate_symbols(symbols, timezone_weights, recommendation_weights):
    #Will return a df that stores scores and advisor count for a symbol
    scores = {}
    weight_kernel = create_weigth_kernel(timezone_weights, recommendation_weights)
    
    for symbol in symbols:
        recommendations = yf.Ticker(symbol).recommendations
        if  recommendations.empty:
            print(f"Couldn't fetch data for {symbol}.")
            continue
        
        recommendations = recommendations.drop('period', axis='columns')
        score, advisor = recommendation_score(recommendations, weight_kernel)
        if score:  
            scores[symbol] = [score, advisor]
        else: 
            print(f"No advisors found for {symbol}.") 
            
    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Scores', 'Advisors'])
    scores_df = scores_df.sort_values(by='Scores', ascending=False)
    
    return scores_df 

In [43]:
scores = evaluate_symbols(symbols, TIMEZONE_WEIGHTS, RECOMMENDATION_WEIGHTS)
scores.head(15)

No advisors found for YHGJ.
No advisors found for XTLB.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FB?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=FB&crumb=6PwKi7QcG9i


Couldn't fetch data for FB.


,Scores,Advisors
BABA,3.139175,194
UNH,3.106796,103
AMZN,3.065116,215
MSFT,2.918782,197
V,2.917722,158
CRM,2.559140,186
XWEL,2.500000,2
BA,2.495575,113
AAPL,2.426829,164
MRK,2.365385,104


In [ ]:
def write_recommendations(scores, file_path, advice_count):
    
    
    